In [85]:
import numpy as np
import pandas as pd
import requests 

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from sklearn.externals import joblib


In [77]:
# Функция сбора вакансий с hh.ru:

def get_id_vacancy(list_name):
    
    list_id_vacancy = []
    for name_vacancy in list_name:       
        url = 'http://api.hh.ru/vacancies/'
        url_params = {
            'User-Agent': UserAgent().chrome,
            'text': name_vacancy,
            'area': 1,
            'page': 0,
            "found": 1,
            "per_page": 50,
            "pages": 40,
            'items': [{}],
        }
        
        #Cобираем id текущей вакансии:
        while True:
            response = requests.get(url, params = url_params)
            if not response.ok:
                break
            else:
                if 'items' in response.json():
                    for i in response.json()['items']:
                        id_vac = i['id']
                        if id_vac != None and id_vac not in list_id_vacancy:
                            list_id_vacancy.append(id_vac)          
                else:
                    id_vac = response.json()['id']
                    if id_vac != None and id_vac not in list_id_vacancy:
                        list_id_vacancy.append(id_vac)
    
    return list_id_vacancy


In [78]:
# Списки запросов для поиска вакансий:

list_name = ['разработчик', 'тестировщик', 'data', 'инженер-программист', 'web-программист', 'программист']


In [ ]:
%%time
list_id_vacancies = get_id_vacancy(list_name)


In [81]:
url = 'http://api.hh.ru/vacancies/'
url_params = {'User-Agent': UserAgent().chrome}
j = list_id_vacancies[300]
result = requests.get(url + j, params = url_params)
result.json()


{'id': '36691474',
 'premium': False,
 'billing_type': {'id': 'standard_plus', 'name': 'Стандарт плюс'},
 'relations': [],
 'name': 'Senior Android разработчик',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 150000, 'to': 200000, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'site': {'id': 'hh', 'name': 'hh.ru'},
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'schedule': {'id': 'remote', 'name': 'Удаленная работа'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p>Друзья, привет.</p> <p>Нам в команду веб-разработчиков Hoodies нужен Android разработчик на зарубежный некоммерческий проект.<br /><br /><strong>Решай глобальные проблемы</strong><br />Пустынная саранча наносит непоправимый вред сельскому хозяйству. 

In [82]:
# По id собираем описания вакансий:

def get_descriptions_vacancies(list_id):
    list_descriptions = []
    url = 'http://api.hh.ru/vacancies/'
    
    for j in list_id:
        description_vacancy = {}
        result = requests.get(url + j, params = url_params)
        if not result.ok:
            break
        else:
            result = result.json()
            list_descriptions.append(result)
            
    return list_descriptions


In [83]:
%%time

vacancies_list = get_descriptions_vacancies(list_id_vacancies)
print(len(vacancies_list))
print(vacancies_list[-1])


5938
{'id': '36722104', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Разработчик T-SQl', 'insider_interview': None, 'response_letter_required': False, 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'noExperience', 'name': 'Нет опыта'}, 'schedule': {'id': 'remote', 'name': 'Удаленная работа'}, 'employment': {'id': 'full', 'name': 'Полная занятость'}, 'department': None, 'contacts': None, 'description': '<p><strong>iFellow</strong> – ИТ компания нового поколения, предлагающая сервисно-ориентированный подход к разработке ПО, тестированию и сопровождению информационных систем для бизнеса. Компания iFellow объединила в себе лучшие российские практики и передовой зарубежный опыт реализации проектов любой сложности для развития информационных технологий

In [84]:
# Сохраним собранные данные:

joblib.dump(vacancies_list, 'vacancies_list.pkl')


['vacancies_list.pkl']